## The canonical annotated pattern

In [ ]:
from typing import Annotated

from pydantic import BaseModel, Field, WithJsonSchema


class Model(BaseModel):
    name: Annotated[str, Field(strict=True), WithJsonSchema({'extra': 'data'})]

In [ ]:
from typing import Annotated

from pydantic import BaseModel, Field


class SomeModel(BaseModel):
    int_list: list[Annotated[int, Field(gt=0)]]
    
from pydantic import ValidationError

try:
    SomeModel(int_list=[-1, 2])
except ValidationError as e:
    print(e)

SomeModel(int_list=[1, 2])

1 validation error for SomeModel
int_list.0
  Input should be greater than 0 [type=greater_than, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


SomeModel(int_list=[1, 2])

In [ ]:
# from typing import Annotated

# from pydantic import BaseModel, Field


# class ImpossibleSomeModel(BaseModel):
#     int_list: list[int] = Field(gt=0)

# ImpossibleSomeModel(int_list=[1, 2])

## Default values

In [11]:
from pydantic import BaseModel, Field


class User(BaseModel):
    # Both fields aren't required:
    name: str = 'John Doe'
    age: int = Field(default=20)

User()

User(name='John Doe', age=20)

In [12]:
from uuid import uuid4

from pydantic import BaseModel, Field


class User(BaseModel):
    id: str = Field(default_factory=lambda: uuid4().hex)

User()

User(id='1a9c4488902e4eaba9f335c5c61a43ca')

In [13]:
#validate default values

from pydantic import BaseModel, Field, ValidationError


class User(BaseModel):
    age: int = Field(default='twelve', validate_default=True)


try:
    user = User()
except ValidationError as e:
    print(e)

1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='twelve', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


In [14]:
from pydantic import BaseModel, EmailStr, Field


class User(BaseModel):
    email: EmailStr
    username: str = Field(default_factory=lambda data: data['email'])


user = User(email='user@example.com')
print(user.username)
#> user@example.com

user@example.com


## Mutable default values (edge case)

In [ ]:
class C:
    x = []
    def add(self, element):
        self.x.append(element)

o1 = C()
o2 = C()
o1.add(1)
o2.add(2)
assert o1.x == [1, 2]
assert o1.x is o2.x   # Do you see the problem ??? Pydantic solves it

In [16]:
from dataclasses import dataclass

try:
    @dataclass
    class D:
        x: list = []      # This code raises ValueError
        def add(self, element):
            self.x.append(element)
except ValueError as e:
    print(e)

mutable default <class 'list'> for field x is not allowed: use default_factory


In [17]:
#Pydantic clones

from pydantic import BaseModel


class Model(BaseModel):
    item_counts: list[dict[str, int]] = [{}]


m1 = Model()
m1.item_counts[0]['a'] = 1
print(m1.item_counts)
#> [{'a': 1}]

m2 = Model()
print(m2.item_counts)
#> [{}]

[{'a': 1}]
[{}]


## Aliases

In [18]:
from pydantic import BaseModel, Field


class User(BaseModel):
    name: str = Field(alias='username')


user = User(username='johndoe')  
assert user.model_dump(by_alias=True)  == {'username': 'johndoe'}

In [19]:
from pydantic import BaseModel, Field


class User(BaseModel):
    name: str = Field(validation_alias='username')


user = User(username='johndoe')  
assert user.model_dump(by_alias=True)  == {'name': 'johndoe'}

In [20]:
from pydantic import BaseModel, Field


class User(BaseModel):
    name: str = Field(serialization_alias='username')


user = User(name='johndoe')  
assert user.model_dump(by_alias=True)  == {'username': 'johndoe'}

In [21]:
from pydantic import AliasGenerator, BaseModel, ConfigDict
from pydantic.alias_generators import to_camel, to_pascal

class Athlete(BaseModel):
    first_name: str
    last_name: str
    sport: str

    model_config = ConfigDict(
        alias_generator=AliasGenerator(
            validation_alias=to_camel,
            serialization_alias=to_pascal,
        )
    )

athlete = Athlete(firstName='John', lastName='Doe', sport='track')
assert athlete.model_dump(by_alias=True) == {'FirstName': 'John', 'LastName': 'Doe', 'Sport': 'track'}

## Strict mode

In [ ]:
from pydantic import BaseModel, Field


class Sales(BaseModel):
    quantity: int = Field(strict=True)
    price: float = Field(strict=False)  


sales = Sales(quantity=5, price='42')  
assert sales.model_dump() == {'quantity': 5, 'price': 42.0}

try:
    sales = Sales(quantity='5', price='42')  
except ValidationError as e:
    print(e)

1 validation error for Sales
quantity
  Input should be a valid integer [type=int_type, input_value='5', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_type


## Extras

In [23]:
from pydantic import BaseModel, Field


class User(BaseModel):
    name: str = Field(repr=True)  
    age: int = Field(repr=False)


user = User(name='John', age=42)
print(user)
assert user.model_dump() == {'name': 'John', 'age': 42}

name='John'


In [24]:
from pydantic import BaseModel, Field, ValidationError


class User(BaseModel):
    name: str = Field(frozen=True)
    age: int


user = User(name='John', age=42)

try:
    user.name = 'Jane'  
except ValidationError as e:
    print(e)

1 validation error for User
name
  Field is frozen [type=frozen_field, input_value='Jane', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/frozen_field


In [25]:
from pydantic import BaseModel, Field


class User(BaseModel):
    name: str
    age: int = Field(exclude=True)


user = User(name='John', age=42) 
assert user.model_dump() == {'name': 'John'}

## Property attributes

In [26]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    def _get_radius(self):
        print("Get radius")
        return self._radius

    def _set_radius(self, value):
        print("Set radius")
        self._radius = value

    def _del_radius(self):
        print("Delete radius")
        del self._radius

    radius = property(
        fget=_get_radius,
        fset=_set_radius,
        fdel=_del_radius,
        doc="The radius property."
    )

In [27]:
circle = Circle(42.0)

circle.radius

Get radius


42.0

In [28]:
circle.radius=1000
circle.radius

Set radius
Get radius


1000

In [29]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """The radius property."""
        print("Get radius")
        return self._radius

    @radius.setter
    def radius(self, value):
        print("Set radius")
        self._radius = value

    @radius.deleter
    def radius(self):
        print("Delete radius")
        del self._radius

In [30]:
circle = Circle(42.0)

circle.radius

Get radius


42.0

In [31]:
circle.radius=1000
circle.radius

Set radius
Get radius


1000

In [32]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """The radius property."""
        print("Get radius")
        return self._radius

In [33]:
circle = Circle(42.0)

circle.radius

Get radius


42.0

In [34]:
try:
    circle.radius=1000
except AttributeError as e:
    print(e)
circle.radius

property 'radius' of 'Circle' object has no setter
Get radius


42.0

In [35]:
from pydantic import BaseModel, computed_field
from warnings import deprecated

class Box(BaseModel):
    width: float
    height: float
    depth: float

    @computed_field
    @property 
    @deprecated("'volume' is deprecated") #just for the sake of it
    def volume(self) -> float:
        return self.width * self.height * self.depth


print(Box.model_json_schema(mode='serialization'))
"""
{
    'properties': {
        'width': {'title': 'Width', 'type': 'number'},
        'height': {'title': 'Height', 'type': 'number'},
        'depth': {'title': 'Depth', 'type': 'number'},
        'volume': {'readOnly': True, 'title': 'Volume', 'type': 'number'},
    },
    'required': ['width', 'height', 'depth', 'volume'],
    'title': 'Box',
    'type': 'object',
}
"""

{'properties': {'width': {'title': 'Width', 'type': 'number'}, 'height': {'title': 'Height', 'type': 'number'}, 'depth': {'title': 'Depth', 'type': 'number'}, 'volume': {'deprecated': True, 'readOnly': True, 'title': 'Volume', 'type': 'number'}}, 'required': ['width', 'height', 'depth', 'volume'], 'title': 'Box', 'type': 'object'}


"\n{\n    'properties': {\n        'width': {'title': 'Width', 'type': 'number'},\n        'height': {'title': 'Height', 'type': 'number'},\n        'depth': {'title': 'Depth', 'type': 'number'},\n        'volume': {'readOnly': True, 'title': 'Volume', 'type': 'number'},\n    },\n    'required': ['width', 'height', 'depth', 'volume'],\n    'title': 'Box',\n    'type': 'object',\n}\n"

In [ ]:
b = Box(width=1, height=2, depth=3)
assert b.model_dump() == {'width': 1.0, 'height': 2.0, 'depth': 3.0, 'volume': 6.0}